# Yolo Import

In [1]:
# OpenCV 
import cv2
import numpy as np
import pandas as pd
# 파일 경로 읽어오기
from glob import glob
import time
# multi thread
import threading
# 이미지 가져오기
import os
from PIL import Image
from tensorflow.keras.preprocessing import image
# Sound 출력
from gtts import gTTS
from IPython.display import Audio

# CNN & OCR Import

In [2]:
# tensorflow : CNN model
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import io
 # OCR Google Cloud api
from google.cloud import vision

In [3]:
# gTTS 음성파일 만들기
# kor_wav = gTTS('주행 불가능 지역입니다', lang = 'ko') 
# kor_wav.save('impossible.wav')

In [4]:
#google cloud platform
# !pip install opencv-contrib-python
# !pip install --upgrade google-cloud-vision

# Prepare : Yolo weight file, CNN Model load, OCR

In [5]:
# CNN 모델 로드
model = load_model('cnn_model/cnn_model2.h5')

In [6]:
# 학습된 yolo weight파일과 cfg 파일 가져오기
net = cv2.dnn.readNet("Yolov4/yolov4-obj_1000.weights", "Yolov4/yolov4-obj.cfg") # Helmet : 0, No Helmet : 1

In [7]:
# obj.names : Helmet/No_Helmet
classes = []
with open("Yolov4/obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))
print(colors)

[[ 57.24295453 122.83130817  32.19239806]
 [ 38.2085446   46.73290706  77.46050625]]


In [8]:
# Use google API - json 파일 가져오기
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'OCR/finalproject-375105-31a5d0d9f5a3.json'
 
client_options = {'api_endpoint': 'eu-vision.googleapis.com'}
client = vision.ImageAnnotatorClient(client_options=client_options)

In [27]:
# ip 주소
url = "https://172.20.10.5:8080/video"
cap = cv2.VideoCapture(url)

fps = cap.get(cv2.CAP_PROP_FPS)  # fps = 25.0

In [10]:
# Empty capture folder
# CNN capture 폴더 처음에 비워줌
for f in glob('capture/*.jpg'):
    os.remove(f)

In [11]:
# # Setting default text
# global passibility,color 
# passibility = "Passibility"
# color = (255, 255, 255)

# Yolov4 Define

In [12]:
# 멀티 쓰레드는 영상과 경고음이 충돌하는 것을 막음
class Worker(threading.Thread):
    def __init__(self):
        self.flag = 0
        super(Worker, self).__init__()

In [13]:
# Yolo 헬멧인식 경고음
def sound():
    display(Audio('sound_file/sounds_warning.wav', autoplay=True))

In [14]:
# CNN 도로인식 경고음
def sound2():
    display(Audio('sound_file/impossible.wav', autoplay=True))

In [15]:
# 사진 찍는 함수
def take_a_photo():
#     url = "https://172.20.10.5:8080/video"
    cap = cv2.VideoCapture(url)
    cv2.namedWindow('camera_screenshot', cv2.WINDOW_AUTOSIZE)

    cnt=0
    while True:
        camera, frame = cap.read()
        if not camera:
            print("Can't read camera")
            break
        cv2.imshow('camera_screenshot', frame)

    #     now = datetime.datetime.now().strftime("%d_%H-%M-%S")
        key = cv2.waitKey(33)

        if key == 26:  # ctrl+z
            cnt=cnt+1
    #         cv2.resize(frame, (416,416))
            cv2.imwrite('take_a_photo/capture'+str(cnt)+'.jpg', frame)
            print('스크린샷 저장')
            # 이미지 크기 조정하여 저장
            files = glob('take_a_photo/capture*.jpg')

            for f in files:
                img = Image.open(f)
                img_resize = img.resize((250,250))
                title, ext = os.path.splitext(f)
                img_resize.save(title+ext)

        elif key == 27:  # 종료시 esc
            break

    cv2.destroyAllWindows() # 스크린샷은 esc 키로 끄기

# OCR Define

In [16]:
# OCR 어린이 보호구역 음성파일 만드는 함수
def get_narrator(text):
    tts = gTTS(text=text, lang='ko')
    filename = 'sound_file/narrator.mp3'
    tts.save(filename)
    
    display(Audio(filename, autoplay=True))
    
# get_narrator("어린이 보호구역입니다")

In [17]:
# OCR 인식중 '어린이'가 인식되는 경우 
def get_text(path):
    result = []
    
    # Load image
    with io.open(path, 'rb') as f:
        content = f.read()
    
    # Get text from image
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    for text in texts[1:]:
        if text.description.isdigit():
            if int(text.description) in [x for x in range(20, 90, 10)]:
                result.append(text.description)
        elif text.description=='어린이':
            result.append(text.description)
            
    return result

In [18]:
def alert_speed_limit(texts):
    message = []
    digit_boolean = list(map(lambda x: x.isdigit(), texts))
    if np.all(digit_boolean):  # 모두 숫자
        for text in texts:
            message.append(f"제한속도 시속 {text}km 구간입니다")
    elif np.any(digit_boolean):  # '어린이' 하나 이상 포함
        for text in np.array(texts)[digit_boolean]:
            message.append(f"어린이 보호구역입니다. 시속 {text}km로 주행하세요")
    else:
        message.append("어린이 보호구역입니다")
        
    for m in message:
        get_narrator(m)  # tts 음성 파일 함수 호출

# CNN Define

In [19]:
def predict_road(model):
    df = pd.DataFrame({'image_path': glob('capture/*.jpg')})
    
    test_datagen = ImageDataGenerator(rescale=1/255.)
    test_generator = test_datagen.flow_from_dataframe(
        df,
        x_col='image_path',
        y_col='None',
        target_size=(250, 250),
        class_mode=None,
        shuffle=False
    )
    
    pred = model.predict_generator(test_generator)
    
    return np.argmax(pred, axis=-1)

In [31]:
# CNN 도로인식 함수
def CNN():    
    global C
    passibility = "Passibility"
    color = (255, 255, 255)

    cap = cv2.VideoCapture(url)

    fps = cap.get(cv2.CAP_PROP_FPS)  # fps = 25.0

    for f in glob('capture/*.jpg'):
        os.remove(f)

    C=[]
    while(True):
        camera, frame = cap.read()

        # # frame resize
        # resize_frame = cv2.resize(frame, (250, 250), interpolation=cv2.INTER_AREA)

        # frame crop & resize
        resize_frame = cv2.resize(frame[:, 120:600], (720, 480), interpolation=cv2.INTER_AREA)

        if resize_frame is not None:

            # Capture 5 images per 1 second
            if (int(cap.get(1)) % 5 == 0):
                # Save image as 001 ~ 005.jpg
                filename = f'{str(len(os.listdir("capture"))%5 + 1).zfill(3)}.jpg'
                cv2.imwrite(f'capture/{filename}', resize_frame)

                # Get text from image (per 5 seconds)
                if (int(cap.get(1)) % 125 == 0):  # 5 * fps
                    if get_text(f'capture/{filename}'):
                        alert_speed_limit(get_text(f'capture/{filename}'))

            if len(os.listdir('capture')) >= 5:
                pred = predict_road(model)

                if np.all(pred == 1):
                    passibility = "Passible"
                    color = (255, 0, 0)
                elif np.all(pred == 0):
                    passibility = "Impassible"
                    color = (0, 0, 255)
                    C.append(np.all(pred == 0))  # Append 1 if all of predictions 0

                for f in glob('capture/*.jpg'):
                    os.remove(f)

            cv2.putText(resize_frame, passibility, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.imshow("Frame", resize_frame)

        # Detect impassible road more than 5 times -> sound()
        if C.count(1) > 4:
            sound2() # 불법주행도로 경고음
            break   # 경고음 출력하면 break -> 계속 진행하려면 없앰

        # To finish program(close windows), press q                     
        q = cv2.waitKey(1)
        if q == ord("q"):
            break
    cv2.destroyAllWindows()

# main

In [43]:
# 창이 뜨면 ctrl + z로 사진 찍기 -> esc 키로 종료 -> 방금 찍은 사진으로 yolo 실행 -> helmet/no_helmet 인식 
# helemt -> CNN 동작 -> q로 종료
take_a_photo()

# 이미지 불러오기
# esc창으로 윈도우 창 닫으면 앞에서 스크린샷 이미지를 불러온다
img=glob('take_a_photo/capture*.jpg')

font = cv2.FONT_HERSHEY_PLAIN
# starting_time = time.time()

frame_id = 0
wList = []
for i in range(10):
    w = Worker()
    wList.append(w)
ti=0
L = []
H = []

while True:
    frame = cv2.imread(sorted(img)[-1])
    frame_id += 1
    height, width, channels = frame.shape
    # Detecting objects
    #416*416 <=== 정확도 조절
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (250, 250), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)   # width of object
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)       #the starting X position of detected object
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))   #percentage
                class_ids.append(class_id)#the name of detected object
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)
    
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.rectangle(frame, (x, y), (x + w, y + 30), color, -1)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 3, (255,255,255), 3)
    
            if class_ids[i] == 1:  # Helmet : 0, No_Helmet : 1
                L.append(class_ids[i])
                
            elif class_ids[i] == 0:
                H.append(class_ids[i])
                
    # 만약 헬멧이 인식되면 프로그램이 끝나고 헬멧을 쓰지 않은 경우는 경고음으로?
    # L.count(1)은 프레임 마다 검사를 했을 때, No_Helmet이라고 판단하는 변수가 몇 이상일 때 경보음을 울릴 것인가 경계값을 정하는 변수 
    # 노헬멧이 인식된 경우
    cv2.imshow("Helmet Detection", frame)
    if(L.count(1)>=1): ##confidence 추가
        print("헬멧 미착용 : " + str(round(confidence, 2)*100) + "%") #헬멧 미착용 인식률
        sound()
        break

    # 헬멧이 인식된 경우 => CNN 모델 추가
    elif(H.count(0)>=1):
        print('헬멧 착용: '+ str(round(confidence, 2)*100) + "%") # 헬멧 착용 인식률
        
        cv2.destroyWindow('Helmet Detection')
        # CNN 모델
        CNN()
        break
        
    key = cv2.waitKey(1)
    if key == ord('q'):  # q or esc 누르면 창 닫힘
        break

# cap.release()
cv2.waitKey()
cv2.destroyAllWindows()

스크린샷 저장
헬멧 착용: 56.00000000000001%
Found 5 validated image filenames.


C:\Users\tjdbs\AppData\Local\Temp\ipykernel_7384\2587802143.py:14: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = model.predict_generator(test_generator)


Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
Found 5 validated image filenames.
